In [8]:
import Pkg
Pkg.activate(@__DIR__)
Pkg.instantiate()
import MathOptInterface as MOI
import Ipopt
import ForwardDiff as FD
import Convex as cvx
import ECOS
using LinearAlgebra
using Plots
using Random
using JLD2
using Test
import MeshCat as mc
using Printf

  Activating environment at `~/Desktop/CMU/Term_2/16745 Optimal Control/OptimalSailboat/minimum time/Project.toml`
Precompiling project...
  ✓ Xorg_libXau_jll
  ✓ PrecompileTools
  ✓ Xorg_libXdmcp_jll
  ✓ libevdev_jll
  ✓ Xorg_libpthread_stubs_jll
  ✓ libpng_jll
  ✓ Libmount_jll
  ✓ Libuuid_jll
  ✓ XZ_jll
  ✓ mtdev_jll
  ✓ Wayland_protocols_jll
  ✓ EpollShim_jll
  ✓ Xorg_xtrans_jll
  ✓ LZO_jll
  ✓ gperf_jll
  ✓ Xorg_libxcb_jll
  ✓ Glib_jll
  ✓ OpenSSL
  ✓ Fontconfig_jll
  ✓ RecipesBase
  ✓ Libtiff_jll
  ✓ eudev_jll
  ✓ Wayland_jll
  ✓ Xorg_libX11_jll
  ✓ Xorg_xcb_util_jll
  ✓ libinput_jll
  ✓ Xorg_libXext_jll
  ✓ Xorg_libxkbfile_jll
  ✓ Xorg_libXfixes_jll
  ✓ Xorg_libXrender_jll
  ✓ ColorVectorSpace
  ✓ Xorg_xcb_util_image_jll
  ✓ Xorg_xcb_util_renderutil_jll
  ✓ Xorg_xcb_util_wm_jll
  ✓ Xorg_xcb_util_keysyms_jll
  ✓ Libglvnd_jll
  ✓ Xorg_libXinerama_jll
  ✓ Xorg_xkbcomp_jll
  ✓ Xorg_libXi_jll
  ✓ Xorg_libXrandr_jll
  ✓ Xorg_libXcursor_jll
  ✓ HTTP
  ✓ Xorg_xcb_util_cursor_jll
  ✓ Xorg

In [45]:
include(joinpath(@__DIR__, "utils","fmincon.jl"))

fmincon (generic function with 1 method)

In [46]:
"""
Converts from cartesian (x,y) to polar (r,θ) coordinates.
"""
function car2pol(x, y)
    r = √(x^2 + y^2)
    θ = atan(y, x)
    return r, θ
end;

"""
Wraps an angle (in radians) to the range [-π, π].
"""
function wrap2pi(θ)
    return θ - 2*π*floor((θ+π)/(2*π))
end;

"""
Converts from a vector of vectors to a matrix.
"""
function vec2mat(X::Vector{Vector{Float64}})::Matrix
    # convert a vector of vectors to a matrix 
    Xm = hcat(X...)
    return Xm 
end;

"""
Converts from a matrix to a vector of vectors.
"""
function mat2vec(Xm::Matrix)::Vector{Vector{Float64}}
    # convert a matrix into a vector of vectors 
    X = [Xm[:,i] for i = 1:size(Xm,2)]
    return X 
end;

In [47]:
"""
true wind velocity (vector field)
"""
function true_wind(xₒ, yₒ)
    ẋ_tw = 1.
    ẏ_tw = 0.
    return ẋ_tw, ẏ_tw
end;

"""
dynamics
"""
function dynamics(x, u, p)
    xₒ, yₒ, θ, v, θ̇ = x
    δᵣ, δₛ = u
    
    # true wind
    ẋ_tw, ẏ_tw = true_wind(xₒ, yₒ)
    a_tw, ψ_tw = car2pol(ẋ_tw, ẏ_tw)
    
    # apparent wind
    ẋ_aw = a_tw * cos(ψ_tw - θ) - v
    ẏ_aw = a_tw * sin(ψ_tw - θ)
    a_aw, ψ_aw = car2pol(ẋ_aw, ẏ_aw)
    
    # forces
    gₛ = p[4] * a_aw * sin(δₛ - ψ_aw)
    gᵣ = p[5] * v^2 * sin(δᵣ)

    # dynamics
    ẋ = [v*cos(θ) + p[1]*a_tw*cos(ψ_tw),
         v*sin(θ) + p[1]*a_tw*sin(ψ_tw),
         θ̇,
         (gₛ*sin(δₛ) - gᵣ*p[11]*sin(δᵣ) - p[2]*v^2)/(p[9]),
         (gₛ*(p[6] - p[7]*cos(δₛ)) - gᵣ*p[8]*cos(δᵣ) - p[3]*θ̇*v)/(p[10])]

    return ẋ
end;

function rk4(params::NamedTuple, x::Vector, u::Vector, dt::Float64)
    # vanilla RK4
    k1 = dt*dynamics(params, x, u)
    k2 = dt*dynamics(params, x + k1/2, u)
    k3 = dt*dynamics(params, x + k2/2, u)
    k4 = dt*dynamics(params, x + k3, u)
    x + (1/6)*(k1 + 2*k2 + 2*k3 + k4)
end

rk4 (generic function with 1 method)

In [48]:
# This function creates some useful indexing tools for Z
# x_i = Z[idx.x[i]]
# u_i = Z[idx.u[i]]
# Feel free to use/not use anything here.
# our Z vector is [x0, u0, x1, u1, …, xN]
function create_idx(nx,nu,N)
    nz = (N-1) * nu + N * nx # length of Z
    x = [(i - 1) * (nx + nu) .+ (1 : nx) for i = 1:N]
    u = [(i - 1) * (nx + nu) .+ ((nx + 1):(nx + nu)) for i = 1:(N - 1)]
    # constraint indexing for the (N-1) dynamics constraints when stacked up
    c = [(i - 1) * (nx) .+ (1 : nx) for i = 1:(N - 1)]
    nc = (N - 1) * nx # (N-1)*nx
    return (nx=nx,nu=nu,N=N,nz=nz,nc=nc,x= x,u = u,c = c)
end

N = 100
tf = 2.0
t_vec = range(0, tf, length = N)
dt = ones(N-1)
nu = 2
nx = 5
Q = diagm([1, 1, 1, 1, 1])
R = diagm([1, 1])
Qf = 0.1 * Q
xic = [15.0, 0.0, 2*π/3, 0.0, 0.0]
xg = [-15.0, 5.0, 2*π/3, 0.0, 0.0]

idx = create_idx(nx,nu,10)

params = (Q = Q,
          R = R,
          Qf = Qf,
          xic = xic,
          xg = xg,
          idx = idx,
          N = N,
          dt = dt,
          tf = tf,
          nx = nx,
          nu = nu,)

(Q = [1 0 … 0 0; 0 1 … 0 0; … ; 0 0 … 1 0; 0 0 … 0 1], R = [1 0; 0 1], Qf = [0.1 0.0 … 0.0 0.0; 0.0 0.1 … 0.0 0.0; … ; 0.0 0.0 … 0.1 0.0; 0.0 0.0 … 0.0 0.1], xic = [15.0, 0.0, 2.0943951023931953, 0.0, 0.0], xg = [-15.0, 5.0, 2.0943951023931953, 0.0, 0.0], idx = (nx = 5, nu = 2, N = 10, nz = 68, nc = 45, x = UnitRange{Int64}[1:5, 8:12, 15:19, 22:26, 29:33, 36:40, 43:47, 50:54, 57:61, 64:68], u = UnitRange{Int64}[6:7, 13:14, 20:21, 27:28, 34:35, 41:42, 48:49, 55:56, 62:63], c = UnitRange{Int64}[1:5, 6:10, 11:15, 16:20, 21:25, 26:30, 31:35, 36:40, 41:45]), N = 41, dt = 0.05, tf = 2.0, nx = 5, nu = 2)

In [51]:
function cost(params, dt::Vector, x::Vector, u::Vector)
    Q, R, Qf = params.Q, Params.R
    J = 0
    for i = 1:N-1
        J += dt[i] * (x[i]' * Q * X[i] +  u[i] * R * u[i])
    end
    J += (x[N]' * Qf * x[N])
    return J
end

function dynamics_constraint(Params, Z)::Vector
    idx, N, dt = params.idx, params.N, params.dt
    p = [0.03, 40, 6000, 200, 1500, 0.5, 0.5, 2, 300, 400, 0.2]
    C = zeros(eltype(Z), idx.nc)
    for i = 1:N-1
        xi = Z[idx.x[i]]
        xip1 = Z[idx.x[i+1]]
        ui = Z[idx.u[i]]

        C[i] = dynamics(xi, ui, p)
    end
    return C
end

function equality_constraint(params::NamedTuple, Z::Vector)::Vector
    xg = params.xg
    N = params.N
    xic = params.xic
    return [Z[idx.x[1]] - xic; dynamics_constraint(params, Z); Z[idx.x[N]] - xg]
end

function inequality_constraint(params, Z)
    return zeros(eltype(Z), 0)
end



inequality_constraint (generic function with 1 method)

In [52]:
x_l = -Inf * ones(nx)
x_u = Inf * ones(nx)

c_l = zeros(0)
c_u = zeros(0)

Z0 = randn(20)

diff_type = :auto # use ForwardDiff.jl

Z = fmincon(cost, equality_constraint, inequality_constraint, 
            x_l, x_u, c_l, c_u, x0, params, diff_type; 
            tol = 1e-6, c_tol = 1e-6, max_iters = 10_000, verbose = true);



MethodError: MethodError: Cannot `convert` an object of type Vector{Float64} to an object of type Float64
Closest candidates are:
  convert(::Type{T}, !Matched::Gray24) where T<:Real at /Users/deniskaanalpay/.julia/packages/ColorTypes/vpFgh/src/conversions.jl:114
  convert(::Type{T}, !Matched::Gray) where T<:Real at /Users/deniskaanalpay/.julia/packages/ColorTypes/vpFgh/src/conversions.jl:113
  convert(::Type{T}, !Matched::Base.TwicePrecision) where T<:Number at twiceprecision.jl:250
  ...